In [14]:
from docx import Document
import pandas as pd
from datetime import datetime
import os
import time
from tqdm import tqdm


In [15]:
def clean_date(cleaned_date_str):
    parsed_date = datetime.strptime(cleaned_date_str, '%B %d, %Y')
    return parsed_date.strftime('%Y-%m-%d')

In [50]:
def extract_articles_to_dataframe(doc_path):
    # Start timer
    start_time = time.time()
    print("Processing document: ", doc_path)
    doc = Document(doc_path)
    count = 0
    articles = []
    total_paragraphs = len(doc.paragraphs)
    while count < total_paragraphs:
        #for para in doc.paragraphs:
        para = doc.paragraphs[count]
        count += 1
        if "Heading 1" in para.style.name: # @Guo: note that Docx files contain metadate on each paragraph, such as the paragraph style, not just the text. I leverage this to obtain the titles.
            article = {}
            correction_appended = False
            article['title'] = para.text.strip()
            article['publisher'] =  doc.paragraphs[count].text
            raw_date = doc.paragraphs[count+1].text
            # Remove trailing weekday if present (e.g., "Tuesday")
            date_parts = raw_date.split()[:3]  # "November 19, 2024"
            cleaned_date_str = " ".join(date_parts).strip(",")
            try:
                article['date'] = clean_date(cleaned_date_str)
            except:
                if doc.paragraphs[count+2].text.strip("\n ") == "Correction Appended":
                    correction_appended = True
            #except ValueError:
             #   continue
            for i in range(3,20):
                text = doc.paragraphs[count+i].text
                if text[:8] == "Section:":
                    section = text[9:]
                    article["section"] = section
                if text[:7] == "Length:":
                    article["length"] = text[8:]
                if text == "Body":
                    count_body_start = count + i + 1
                    break
            # Extract body
            body = []
            new_count = count_body_start
            while text.strip("\n ").lower() != "end of document":
                text = doc.paragraphs[new_count].text
                body.append(text)
                new_count += 1
            article["body"] = "".join(body[:-1])

            # Handle correction
            if correction_appended:
                date_parts  = article["body"].split("Correction-Date: ")[-1].split()[:3]
                cleaned_date_str = " ".join(date_parts).strip(",")
                article['date'] = clean_date(cleaned_date_str)
            count = new_count
            article["correction_appended"] = correction_appended
            articles.append(article)
    print("Number of articles: ", len(articles))

    df = pd.DataFrame(articles)
    df["source_file"] = os.path.basename(doc_path)

    # End timer
    end_time = time.time()

    # Calculate the difference
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time / 60:.2f} minutes")
    return df

In [51]:
import glob
docx_files = glob.glob(os.path.join('..', 'data', 'raw', '*', '*.DOCX'))
print("Files to process: ", len(docx_files))

Files to process:  88


In [ ]:
doc

In [ ]:
df_articles = pd.DataFrame()
for doc_path in docx_files:
    df = extract_articles_to_dataframe(doc_path)
    df_articles = pd.concat([df_articles, df])
    print("Total articles: ", df_articles.shape[0])

Processing document:  ../data/raw/Other publishers/Files (500) (21).DOCX


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1055db860>>
Traceback (most recent call last):
  File "/Users/laura/venvs/python3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [33]:
df_articles = pd.DataFrame()
for doc_path in docx_files:
    df = extract_articles_to_dataframe(doc_path)
    df_articles = pd.concat([df_articles, df])
    print("Total articles: ", df_articles.shape[0])

Processing document:  ../data/raw/Other publishers/Files (500) (21).DOCX
Number of articles:  500
Elapsed time: 13.77 minutes
Total articles:  500
Processing document:  ../data/raw/Other publishers/Files (500) (2).DOCX


UnboundLocalError: cannot access local variable 'new_count' where it is not associated with a value

In [46]:
doc_path = "../data/raw/Other publishers/Files (500) (2).DOCX"
df = extract_articles_to_dataframe(doc_path)

Processing document:  ../data/raw/Other publishers/Files (500) (2).DOCX
Number of articles:  500
Elapsed time: 10.02 minutes


In [47]:
df.shape

(500, 8)

In [48]:
print(df.body[0])

South Minneapolis environmental activists have raised - and the city has verified - $3.7 million in donations to buy 7.6 acres of city-owned property at E. 28th Street and Longfellow Avenue in the East Phillips neighborhood.The property contains a former Roof Depot warehouse, which the group East Phillips Neighborhood Institute (EPNI) intends to redevelop into an indoor farm.The city gave EPNI until Wednesday to provide proof of having the $3.7 million in order to complete the sale. The total fair market value of the site is $11.4 million. The Legislature appropriated $2 million for the sale last session and pledged to secure another $5.7 million next year."It's awesome, it really is," said Dean Dovolis, founder of DJR Architecture and an EPNI board member. "The community came through and this is very significant."Erik Hansen, Minneapolis' director of economic policy and development, confirmed the city accepted the guaranty on Wednesday afternoon.Dovolis said EPNI will now start the en

In [49]:
print(df.body[1])

A majority of Americans support higher pay for auto workers who are on strike against Detroit's Big Three carmakers, although approval of the workers' other demands is more mixed, according to a poll from The Associated Press-NORC Center for Public Affairs Research.The poll found that 36% of Americans sympathize with the workers in their dispute with the automakers, 9% support the automakers, and the rest back both or neither. Support for the autoworkers fell short of the 55% support for striking Hollywood writers and actors in an AP-NORC poll conducted last month. Still, the new poll adds to evidence of U.S. support for labor unions during a year marked by strikes in Hollywood, a walkout that was narrowly averted by Teamsters at United Parcel Service, and now the picket lines outside auto plants. In the new AP-NORC survey, 51% say labor unions help U.S. workers while only 15% say they hurt working people. About one-third say unions help the U.S. economy, while 22% say they damage the 

In [41]:
# Start timer
start_time = time.time()
print("Processing document: ", doc_path)
doc = Document(doc_path)
count = 0
articles = []
while count < len(doc.paragraphs):
    #for para in doc.paragraphs:
    para = doc.paragraphs[count]
    count += 1
    if "Heading 1" in para.style.name: # @Guo: note that Docx files contain metadate on each paragraph, such as the paragraph style, not just the text. I leverage this to obtain the titles.
        article = {}
        correction_appended = False
        article['title'] = para.text.strip()
        article['publisher'] =  doc.paragraphs[count].text  
        raw_date = doc.paragraphs[count+1].text
        # Remove trailing weekday if present (e.g., "Tuesday")
        date_parts = raw_date.split()[:3]  # "November 19, 2024"
        cleaned_date_str = " ".join(date_parts).strip(",")
        try:
            article['date'] = clean_date(cleaned_date_str)
        except:
            if doc.paragraphs[count+2].text.strip("\n ") == "Correction Appended":
                correction_appended = True
        #except ValueError:
         #   continue
        for i in range(3,9):
            print(i)
            text = doc.paragraphs[count+i].text
            if text[:8] == "Section:":
                section = text[9:]
                article["section"] = section
            if text[:7] == "Length:":
                article["length"] = text[8:]
            if text == "Body":
                print("Body")
                body = []
                new_count = count + i + 1
                continue
        while text.strip("\n ").lower() != "end of document":
            text = doc.paragraphs[new_count].text
            body.append(text)
            new_count += 1
        article["body"] = "".join(body[:-1])
        if correction_appended:
            date_parts  = article["body"].split("Correction-Date: ")[-1].split()[:3]
            cleaned_date_str = " ".join(date_parts).strip(",")
            article['date'] = clean_date(cleaned_date_str)
        count = new_count # why not just use the count variable?
        article["correction_appended"] = correction_appended
        articles.append(article)
print("Number of articles: ", len(articles))

df = pd.DataFrame(articles)
df["source_file"] = os.path.basename(doc_path)

# End timer
end_time = time.time()

# Calculate the difference
elapsed_time = end_time - start_time
#print(f"Elapsed time: {elapsed_time:.4f} seconds")
print(f"Elapsed time: {elapsed_time / 60:.2f} minutes")

Processing document:  ../data/raw/Other publishers/Files (500) (2).DOCX
3
4
5
6
7
8


IndexError: list index out of range

In [42]:
text

'Highlight:\xa0Environmental activists have sought to redevelop it as an indoor farm.'

In [43]:
article

{'title': "Group's $3.7M to buy Roof Depot",
 'publisher': 'Star Tribune (Minneapolis, MN)',
 'date': '2023-11-09',
 'section': 'NEWS; Pg. 1B',
 'length': '461 words'}

In [38]:
new_count

26881

In [39]:
len(doc.paragraphs)

22204

In [40]:
len(articles)

0